# Comparativa de las técnicas de modelado de la previsión de la demanda para dimensionar el stock en el almacén.


### Universitat Oberta de Catalunya - Ciència de Dades
### Jordi Puiggròs Cuadras - jpuiggrosc@uoc.edu

#### En este notebook se procederá a la obtención, preparación, análisis estadístico y visual del juego de datos para el justificar la memória del proyecto. Así mismo, se realizaran los modelos y se evaluará la calidad de estos:

<ol>
<li>Preparación del set de datos</li>
<li>Modelado</li>
<li>Evaluación de la calidad</li>
</ol>


## 1. Preparación del set de datos
#### Se cargan los datos a partir de los ficheros csv y se preparan para su análisis estadístico y visual. El objetivo es tener un único dataframe final, con los datos preparados, transformados y que puedan ser usados por los distintos modelos de ML que se quieren aplicar

In [ ]:
# Importem llibreries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
from matplotlib import pyplot
plt.style.use('seaborn-darkgrid')

### 1.1 Cargamos los ficheros
#### Existen 4 ficheros: Ventas, Calendario, Promociones y Stocks. Se usaran como columnas indexadoras la fecha y los sku.

In [ ]:
# Importem els datasets
dsVentas = pd.read_csv('.\DATA\Venta.csv',  sep = ';' )
dsCalendari = pd.read_csv('.\DATA\Calendari.csv',  sep = ';')
dsPromos = pd.read_csv('.\DATA\Promos.csv',  sep = ';')
dsStock = pd.read_csv('.\DATA\Stock.csv',  sep = ';')

# Calendari: dates en format date
dsCalendari.insert(0, "Data",pd.to_datetime(dsCalendari['fecha'],format='%Y%m%d'), True)
dsCalendari = dsCalendari.drop('fecha', 1) #0-> rows, 1->cols

# Ventes: dates en format date
dsVentas.insert(0, "Data",pd.to_datetime(dsVentas['fecha'],format='%Y%m%d'), True)
dsVentas = dsVentas.drop('fecha', 1) #0-> rows, 1->cols

# Promos: dates en format date
dsPromos.insert(0, "Data_ini",pd.to_datetime(dsPromos['fecha ini'],format='%Y%m%d'), True)
dsPromos.insert(1, "Data_fin",pd.to_datetime(dsPromos['fecha fin'],format='%Y%m%d'), True)
dsPromos = dsPromos.drop('fecha ini', 1) #0-> rows, 1->cols
dsPromos = dsPromos.drop('fecha fin', 1) #0-> rows, 1->cols

# Stock: dates en format date
dsStock.insert(0, "Data",pd.to_datetime(dsStock['fecha'],format='%Y%m%d'), True)
dsStock = dsStock.drop('fecha', 1) #0-> rows, 1->cols

In [ ]:
print("1.- Ventas")
print(dsVentas.describe())

print("2.- Calendario")
print(dsCalendari.describe())

print("3.- Promociones")
print(dsPromos.describe())

print("4.- Stocks")
print(dsStock.describe())

In [ ]:
# Fusionem el fitxer del calendari amb el de ventes
dfCalendariVentas = pd.merge(dsCalendari, dsVentas, on=['sku','Data'], how='left')
dfCalendariVentas

### 1.2 Fusionamos los ficheros
#### El objetivo final de este apartado es conseguir un único dataframe con todos los datos útiles para realizar el análisis y el modelado

In [ ]:
# Preparem el fitxer de promocions per fusionar-lo
out = pd.merge(dfCalendariVentas,dsPromos, on='sku', how='left')
m = out['Data'].between(out['Data_ini'], out['Data_fin'])
Promocions = out[m]
#Font: https://tutorialmeta.com/question/is-there-a-way-to-merge-on-interval-index-and-another-column-value-in-pandas

In [ ]:
# Fusionem les promocions amb el dataset acumulatiu
dfCalendariVentasPromocion = pd.merge(dfCalendariVentas, Promocions, on=['sku','Data'], how='left')
dfCalendariVentasPromocion

# Creem una nova la columna: "Promoción" amb 0 o 1 segons correspongui
dfCalendariVentasPromocion.insert(5, "Promocion",np.where(dfCalendariVentasPromocion.Data_ini.isnull(), 0, 1), True)

# Eliminem les columnes sobrants
dfCalendariVentasPromocion = dfCalendariVentasPromocion.drop('bolOpen_y', 1) #0-> rows, 1->cols
dfCalendariVentasPromocion = dfCalendariVentasPromocion.drop('bolHoliday_y', 1) #0-> rows, 1->cols
dfCalendariVentasPromocion = dfCalendariVentasPromocion.drop('udsVenta_y', 1) #0-> rows, 1->cols

# Renombrem columnes
dfCalendariVentasPromocion = dfCalendariVentasPromocion.rename(columns={"bolOpen_x": "bolOpen", "bolHoliday_x": "bolHoliday", "udsVenta_x":"udsVenta"})

dfCalendariVentasPromocion

In [ ]:
# Afegim els valors del dataset Stock
dfCalendariVentasPromocionStock = pd.merge(dfCalendariVentasPromocion, dsStock, on=['sku','Data'], how='left')
dfCalendariVentasPromocionStock

### 1.3 Transformación del dataset
#### Una vez contamos con un único dataframe, lo trabajaremos para transformarlo en un dataframe útil y sin ruido de fondo. Transformando los valores NaN, quitando las fechas de la serie temporal sin valores (tanto por arriba como por abajo), y reconstruiendo algunos datos basándonos en alguna reglas de calidad básicas que hemos extraido a partir de una primera exploración de los ficheros (por ejemplo, reconstrucción de las ventas cuando el stock sea zero como detallamos en los siguientes apartados)

In [ ]:
# Valors de ventes NA substituïm per 0
dfCalendariVentasPromocionStock['udsVenta'] = dfCalendariVentasPromocionStock['udsVenta'].fillna(0)

# Ordenem i mostrem
dfCalendariVentasPromocionStock.sort_values(by=['Data','sku'], inplace=True)
dfCalendariVentasPromocionStock

In [ ]:
# Mirem quines són la primera i la última data amb ventes
dfMaxMin = dfCalendariVentasPromocionStock.loc[dfCalendariVentasPromocionStock['udsVenta'] > 0]

datamax = dfMaxMin['Data'].max()
datamin = dfMaxMin['Data'].min()

# Filtrem el dataset per agafar només dades amb ventes
df = dfCalendariVentasPromocionStock.loc[(dfCalendariVentasPromocionStock['Data'] >= datamin) & (dfCalendariVentasPromocionStock['Data'] <= datamax)]
df

#### Análisis y tratamiento de atípicos

Por la pandemia, detectamos periodos de ventas a zero, que normalizaremos con un valor medio
Análisis de datos atípicos (datos fuera de los límites de control, gráficos box plot...)
Análisis de periodos atípicos: durante la pandemia y confinamiento, la venta se fue a cero (qué hacer con esas semanas? lo mejor es reemplazar por datos promedio)

In [ ]:
# Reglas de Calidad: 
#Cuando bolOpen=0 y bolHoliday=0 -> Las ventas siempre son 0
df[(df['bolOpen']==0) & (df['bolHoliday']==0) & (df['udsVenta']>0) ]

In [ ]:
# Quan Stock=0 i les ventes=0, però no estem en la situació anterior: s'han de reconstruïr
#df[(df['udsStock']==0)  & (df['udsVenta']==0) & ((df['bolOpen']!=0) | (df['bolHoliday']!=0))]
df[(df['udsVenta']<=0) & ((df['bolOpen']!=0) | (df['bolHoliday']!=0))]

In [ ]:
# Interpolem per a cada un dels 50 SKU
#myList=range(1,51)
#for i in myList:
#    df[df['sku'] == i] = df[df['sku'] == i].interpolate(method ='linear', limit_direction ='forward') 


In [ ]:
# Pasos per reconstruïr les ventes:

# En primer lugar seleccionamos los registros a reconstruir:
#mask = ((df['udsStock']==0)  & (df['udsVenta']==0) & ((df['bolOpen']!=0) | (df['bolHoliday']!=0)))
mask = ((df['udsVenta']<=0) & ((df['bolOpen']!=0) | (df['bolHoliday']!=0)))

myList=range(1,51)
for i in myList:
    aux = df[df['sku']==i]
    mean = aux['udsVenta'].mean()
    df.loc[mask & (df['sku']==i),'udsVenta']= mean


In [ ]:
# Existeixen varias formas de reconstruir las ventas (los métodos por interpolación obtienen mejores resultados (1)):
#dfReconstruido = df[['Data','sku','udsVenta']]
#dfReconstruido = dfReconstruido.interpolate();

# Mezclamos los valores interpolados en un único dataset
#dfReconstruido = pd.merge(df, dfReconstruido, on=['sku','Data'], how='left')

# Comprobamos que efectivamente hemos reconstruido los valores de venta con Stock=0
#dfReconstruido[dfReconstruido['udsStock']==0 ]

# Guardamos un dataset nuevo con todas las variables que queremos usar:

#df2 = dfReconstruido[['Data','sku','bolOpen','bolHoliday','Promocion','udsStock','udsVenta_y']]

# Renombrem columna
#df2 = df2.rename(columns={"udsVenta_y": "udsVenta"})
#df2
df = df[['Data','sku','bolOpen','bolHoliday','Promocion','udsStock','udsVenta']]

Font: 
(1) Medium. Imputing the Time-Series Using Python. url: https://drnesr.medium.com/filling-gaps-of-a-time-series-using-python-d4bfddd8c460

### 1.3 Análisis estadístico
#### El objetivo es realizar un primer análisis del dataset que hemos preparado en los apartados anteriores

##### Vamos a comenzar obteniendo un histograma de frecuencias para conocer la distribución histórica que han tenido las ventas. En este histograma también vamos a añadir la función de densidad de probabilidad (FDP) de una distribución normal. Esto nos permitirá saber si la distribución histórica de los retornos se ajusta, o no, a una distribución normal

In [ ]:
#df = df2

import seaborn as sns
from scipy import stats
from scipy.stats import norm

# Dibujamos el histograma de frecuencias
fig,ax = plt.subplots(figsize=(16,5),dpi=100)

sns.set(color_codes = True)
ax = sns.distplot(df['udsVenta'], bins=100, kde=False, fit=stats.norm, color='green')

# Obtenemos los parámetros ajustados de la distribución normal utilizados por SNS
(mu, sigma) = stats.norm.fit(df['udsVenta'])

# Configuramos la gráfica
plt.title('Distribución de las unidades demandadas en el periodo analizado', fontsize = 16)
plt.ylabel('Frecuencia')
plt.legend(["Distribución normal. fit ($\mu=${0:.2g}, $\sigma=${1:.2f})".format(mu,sigma),"Distribución udsVenta"])

##### Aquí podemos ver que nuestra distribución de udsVenta tiene cierta similitud con una distribución normal, sin embargo, esta no llega a ajustarse perfectamente. En realidad, las series de udsVenta que nos encontramos rara vez, si no nunca, se ajustan perfectamente a una distribución normal. En general tienden a mostrar valores extremos que se desvían de su media con una probabilidad más alta de lo que se espera en una distribución normal. Lo cual produce distribuciones con colas largas y por ende mayor probabilidad de sufrir riesgo de cola (Tail Risk).

##### A continuación vamos a obtener la estadística descriptiva de usdVenta. Aquí hemos añadido los estadísticos que considero más importantes. Dentro del código encontraremos comentarios que nos indican para qué sirven las distintas instrucciones:

In [ ]:
# Obtenemos promedio, desviación típica, máximo, mínimo y número de datos analizados para el valor a predecir udsVenta:
print('> Media:', '%.6s' % (df['udsVenta'].mean()))
print('> Desviación Típica:', '%.6s' % (df['udsVenta'].std(ddof=1)))
print('> Mínimo valor:', '%.6s' % ( df['udsVenta'].min()))
print('> Máximo valor:', '%.6s' % ( df['udsVenta'].max()))
print('> Registros analizados:', '%.6s' % (df['udsVenta'].count()))

In [ ]:
sns.distplot(df['udsVenta'])
plt.figure(figsize=(16,5), dpi=100)
plt.show()

In [ ]:
# Añadimos el dia de la semana i el mes del año

#Análisis de correlación de variables (tipo correlation matrix).
import seaborn as sn
df['DayWeek'] = df['Data'].dt.weekday
df['Month'] = df['Data'].dt.month
dfMatrix = pd.DataFrame(df,columns=['sku','bolOpen','bolHoliday','Promocion', 'udsVenta','DayWeek','Month'])


corrMatrix = dfMatrix.corr()
sn.heatmap(corrMatrix, annot=True)
plt.figure(figsize=(16,5), dpi=100)
plt.show()


In [ ]:
# Scatter plot:

sns.set()
cols = ['sku','bolOpen','bolHoliday','Promocion', 'udsVenta','DayWeek','Month']
sns.pairplot(df[cols], size = 2.5)
plt.figure(figsize=(16,5), dpi=100)
plt.show();

In [ ]:
# Diagrama de cajas sku/udsVenta:

var = 'sku'
data = pd.concat([df['udsVenta'], df[var]], axis=1)
f, ax = plt.subplots(figsize=(16, 5), dpi=100)
fig = sns.boxplot(x=var, y="udsVenta", data=data)
fig.axis(ymin=0, ymax=200);


In [ ]:
def replace_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df.copy()
    outliers = ~df_out[col_name].between(fence_low, fence_high, inclusive=False)
    df_out.loc[outliers, col_name] = df_out.loc[~outliers, col_name].mean()
    return df_out

appended_data = []
myList=range(1,51)
for i in myList:
    data = replace_outlier(df[df['sku'] == i], 'udsVenta')
    appended_data.append(data[data['sku'] == i])
  
    
    

In [ ]:
df_SinOutliers = pd.concat(appended_data)
df_SinOutliers

In [ ]:
df=df_SinOutliers

In [ ]:
# Diagrama de cajas sku/udsVenta:

var = 'sku'
data = pd.concat([df['udsVenta'], df[var]], axis=1)
f, ax = plt.subplots(figsize=(16, 5), dpi=100)
fig = sns.boxplot(x=var, y="udsVenta", data=data)
fig.axis(ymin=0, ymax=200);

### 1.4 Análisis visual
#### El objetivo es realizar un análisis visual del dataset que hemos preparado en los apartados anteriores


In [ ]:
# Preparamos una función per crear las gràficas

def plot_df(df,x,y1,y2, title="Titol", xlabel='X', y1label='Y', y2label='Y', dpi=100):  
    fig,ax = plt.subplots(figsize=(16,5),dpi=dpi)
    ax.plot(x, y1, color='tab:red')
    ax.set_ylabel(y1label,color="red",fontsize=14)
    
    # twin object for two different y-axis on the sample plot
    ax2=plt.twinx()
    ax2.plot(x, y2, color='tab:blue')
    ax2.set_ylabel(y2label,color="blue",fontsize=14)
   
    plt.gca().set(title=title, xlabel=xlabel) 
    plt.show()    

In [ ]:
# Grafiquem les séries sobreposant l'Stock para cada uno de los SKUS:
by_label = df.groupby('sku')
for name, group in by_label:
    plot_df(df, x=group['Data'], y1=group['udsVenta'], y2=group['udsStock'], title='sku:'+str(name), xlabel='Data', y1label='Unitats Venudes', y2label='Unitats Stock')
#plt.legend()
plt.show

## 2. Modelado

#### A continuación usaremos dos modelos tradicionales y dos modelos de ML para realizar predicciones forecasting a partir del juego de datos preparado en los apartados anteriores

In [ ]:
# Seleccionamos las columnas que queremos usar en los modelos:
df = df[['Data','sku','bolOpen','bolHoliday','Promocion', 'udsVenta','DayWeek','Month']]

### 2.1 Modelo 1: Exponential Smoothing


In [ ]:
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from sklearn.metrics import mean_squared_error
import math
from math import sqrt
import numpy as np

def calcular_mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    c = np.divide((actual - pred),  actual, out=np.zeros_like((actual - pred)), where= actual!=0)
    return np.nanmean(np.abs(c)) * 100
    
def Predict_SimpleExpSmoothing(df, sku, alpha):
    
    # Split Train Test
    total_size=len(df)
    split = 10392 / 11856
    train_size=math.floor(split*total_size)

    # Test y entrenamiento
    train=df.head(train_size)
    test=df.tail(len(df) - train_size)
    
    # Create prediction table
    y_hat = test.copy()
    fit = SimpleExpSmoothing(np.asarray(train['udsVenta'])).fit(smoothing_level=alpha,optimized=False)
    y_hat['SES'] = fit.forecast(len(test))

    # Metriques de qualitat , initialization_method='estimated'
    rmse = sqrt(mean_squared_error(test.udsVenta, y_hat.SES))
    mape = calcular_mape(test.udsVenta,y_hat.SES)
    
    # Plotting data
    plt.figure(figsize=(12,8), dpi=100)
    plt.plot(train.Data, train['udsVenta'], label='Train')
    plt.plot(test.Data,test['udsVenta'], label='Test')
    plt.plot(y_hat.Data,y_hat['SES'], label='SES')
    plt.legend(loc='best')
    plt.title("Simple Exponential Smoothing (SES)")
    plt.show()
    
    # Medidas a Mostrar
    print("RMSE (sku:" + str(sku) +" alpha:" + str(alpha) +" ) = " + str(rmse) + " MAPE = " + str(mape))
    
    return rmse, mape

# Font: 
# https://github.com/tristanga/Machine-Learning/blob/master/Time%20Series%20Forecasting/Simple%20Exponential%20Smoothing.ipynb

In [ ]:
# Ejecutamos la previsión con SE usando distintos niveles de smoothing_level
Results_SE = pd.DataFrame(columns=['sku','rmse02', 'rmse06','rmse08', 'mape02', 'mape06','mape08'])
myList=range(1,51)

for i in myList:
    #print(df[df['sku'] == i])
    rmse_02, mape02 = Predict_SimpleExpSmoothing(df[df['sku'] == i], i, 0.2)
    rmse_06, mape06 = Predict_SimpleExpSmoothing(df[df['sku'] == i], i, 0.6)
    rmse_08, mape08 = Predict_SimpleExpSmoothing(df[df['sku'] == i], i, 0.8)
    Results_SE = Results_SE.append({'sku': str(i), 'rmse02': rmse_02,'rmse06': rmse_06,'rmse08': rmse_08, 'mape02': mape02, 'mape06': mape06, 'mape08': mape08}, ignore_index=True)
    

In [ ]:
# Agafem el millor resultat de tots

Results_SE['rmse'] = Results_SE[['rmse02','rmse06','rmse08']].min(axis=1)
Results_SE['mape'] = Results_SE[['mape02','mape06','mape08']].max(axis=1)
Results_SE


### 2.2 Modelo 2: Moving Averages


In [ ]:
from statsmodels.tsa.arima_model import ARMA

def Predict_ARMA(df, sku):
    
    # Split Train Test
    total_size=len(df)
    split = 10392 / 11856
    train_size=math.floor(split*total_size)

    # Test y entrenamiento
    train=df.head(train_size)
    test=df.tail(len(df) - train_size)
    
    # Create prediction table
    y_hat = test.copy()
    model_fitted = ARMA(np.asarray(train['udsVenta']), order=(0, 1) ).fit()
    #y_hat['ARMA'] = fit.forecast(len(test))

#model_fitted.summary()

    y_hat['ARMA'] = model_fitted.predict(start=len(train), end=len(train) + len(test)-1, dynamic= True)
    
    # Metriques de qualitat
    rmse = sqrt(mean_squared_error(test.udsVenta, y_hat.ARMA))
    mape = calcular_mape(test.udsVenta,y_hat.ARMA)
        
    # Plotting data
    plt.figure(figsize=(12,8), dpi=100)
    plt.plot(train.Data, train['udsVenta'], label='Train')
    plt.plot(test.Data,test['udsVenta'], label='Test')
    plt.plot(y_hat.Data,y_hat['ARMA'], label='ARMA')
    plt.legend(loc='best')
    plt.title("ARMA")
    plt.show()
    
    # Medidas a Mostrar
    print("RMSE (sku:" + str(sku)+" ) = "+ str(rmse) + " MAPE = " + str(mape))
    
    return rmse, mape

# https://www.projectpro.io/recipes/forecast-moving-averages-for-time-series
#https://puneet166.medium.com/time-series-forecasting-how-to-predict-future-data-using-arma-arima-and-sarima-model-8bd20597cc7b

In [ ]:
# Ejecutamos la previsión con SE usando distintos niveles de smoothing_level
Results_ARMA = pd.DataFrame(columns=['sku','rmse'])
myList=range(1,51)

for i in myList:
    #print(df[df['sku'] == i])
    rmse, mape = Predict_ARMA(df[df['sku'] == i], i)
    Results_ARMA = Results_ARMA.append({'sku': str(i), 'rmse': rmse, 'mape': mape}, ignore_index=True)
    

In [ ]:
Results_ARMA

### 2.3 Modelo 3: SVM




In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

def Predict_SVR(df, sku):

    # Split Train Test
    total_size=len(df)
    split = 10392 / 11856
    train_size=math.floor(split*total_size)

    # Test y entrenamiento
    train=df.head(train_size)
    test=df.tail(len(df) - train_size)

    y_hat = test.copy()
    
    #sc_X = StandardScaler()
    #sc_y = StandardScaler()
    sc_X = preprocessing.MinMaxScaler()
    sc_y = preprocessing.MinMaxScaler()
    
    X_train = sc_X.fit_transform(train[["bolOpen","bolHoliday","Promocion",'DayWeek']])
    y_train = sc_y.fit_transform(train[["udsVenta"]])
    X_test  = sc_X.fit_transform(test[["bolOpen","bolHoliday","Promocion",'DayWeek']])
    y_test  = sc_y.fit_transform(test[["udsVenta"]])

    
    param_grid = {"C": np.linspace(10**(-2),10**3,100),
             'gamma': np.linspace(0.0001,1,20)}
    mod = SVR(epsilon = 0.1,kernel='rbf')
    model = GridSearchCV(estimator = mod, param_grid = param_grid,
                                   scoring = "neg_mean_squared_error",verbose = 0)

    best_model = model.fit(X_train, y_train.ravel())


    #prediction
    #predicted_tr = model.predict(X_train)
    predicted = model.predict(X_test)

    # inverse_transform because prediction is done on scaled inputs
    #predicted_tr = sc_y.inverse_transform(predicted_tr)
    y_hat['SVR'] = sc_y.inverse_transform(predicted.reshape(-1,1))

    # Metriques de qualitat
    rmse = sqrt(mean_squared_error(test.udsVenta, y_hat.SVR))
    mape = calcular_mape(test.udsVenta,y_hat.SVR)
    
    # Plotting data
    plt.figure(figsize=(12,8), dpi=100)
    plt.plot(train.Data, train['udsVenta'], label='Train')
    plt.plot(test.Data,test['udsVenta'], label='Test')
    plt.plot(y_hat.Data,y_hat['SVR'], label='SVR')
    plt.legend(loc='best')
    plt.title("SVR")
    plt.show()

    print("RMSE (sku:" + str(sku)+" ) = "+ str(rmse) + " MAPE = " + str(mape))
    print(best_model.best_params_)
    
    return rmse,mape

In [ ]:
# Ejecutamos la previsión con SRV
Results_SVR = pd.DataFrame(columns=['sku','rmse'])
myList=range(1,51)

for i in myList:
    #print(df[df['sku'] == i])
    rmse, mape = Predict_SVR(df[df['sku'] == i], i)
    Results_SVR = Results_SVR.append({'sku': str(i), 'rmse': rmse, 'mape': mape}, ignore_index=True)

#Font: https://medium.com/pursuitnotes/support-vector-regression-in-6-steps-with-python-c4569acd062d
#https://www.analyticsvidhya.com/blog/2020/03/support-vector-regression-tutorial-for-machine-learning/

### 2.4 Modelo 4: RNN

In [ ]:
import tensorflow as tf
from tensorflow import keras 

def Predict_LSTM(df, sku):

    # Split Train Test
    total_size=len(df)
    split = 10392 / 11856
    train_size=math.floor(split*total_size)

    # Test y entrenamiento
    train=df.head(train_size)
    test=df.tail(len(df) - train_size)

    y_hat = test.copy()
    
    sc_X = StandardScaler()
    sc_y = StandardScaler()

    X_train = sc_X.fit_transform(train[["bolOpen","bolHoliday","Promocion",'DayWeek']])
    y_train = sc_y.fit_transform(train[["udsVenta"]])
    X_test  = sc_X.fit_transform(test[["bolOpen","bolHoliday","Promocion",'DayWeek']])
    y_test  = sc_y.fit_transform(test[["udsVenta"]])

    np.random.seed(42)
    #tf.random.set_seed(42)

    lstm_model = keras.models.Sequential([
        keras.layers.LSTM(20, return_sequences=True, input_shape=[None, 1]),
        keras.layers.LSTM(20, return_sequences=True),   ###
        keras.layers.LSTM(20),
        keras.layers.Dense(1)   #### Note - Dense layer with one output ( One day prediction )
    ])

    optimizer = keras.optimizers.Adam(lr=0.0005)
    lstm_model.compile(loss="mse", optimizer=optimizer)
    lstm_model.summary()

    #### Early stop the training if there is no improvement in val_loss for 5 epochs. Save the best model based on val_loss.
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1)
    mc = keras.callbacks.ModelCheckpoint('best_model_lstm.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

    history_lstm = lstm_model.fit(X_train, y_train, epochs=80, batch_size=32,
                                  validation_data=(X_test, y_test), callbacks=[early_stopping, mc])
    lstm_model = keras.models.load_model('best_model_lstm.h5')

    predicted = lstm_model.predict(X_test)

    # inverse_transform because prediction is done on scaled inputs
    y_hat['LSTM'] = sc_y.inverse_transform(predicted)

    # Metriques de qualitat
    rmse = sqrt(mean_squared_error(test.udsVenta, y_hat.LSTM))
    mape = calcular_mape(test.udsVenta,y_hat.LSTM)

    # Plotting data
    plt.figure(figsize=(12,8), dpi=100)
    plt.plot(train.Data, train['udsVenta'], label='Train')
    plt.plot(test.Data,test['udsVenta'], label='Test')
    plt.plot(y_hat.Data,y_hat['LSTM'], label='LSTM')
    plt.legend(loc='best')
    plt.title("LSTM")
    plt.show()

    print("RMSE (sku:" + str(sku)+" ) = "+ str(rmse) + " MAPE = " + str(mape))
    #print(best_model.best_params_)
    
    return rmse, mape

#Font: https://github.com/raja-surya/Time-Series-RNN/blob/main/Web-Traffic-Time-Series-Forecasting-RNN-LSTM.ipynb
#https://medium.com/geekculture/time-series-forecast-using-deep-learning-adef5753ec85
# https://stackoverflow.com/questions/53890194/time-series-forcasting-with-svr


In [ ]:
# Ejecutamos la previsión con SRV
Results_LSTM = pd.DataFrame(columns=['sku','rmse'])
myList=range(1,51)

for i in myList:
    #print(df[df['sku'] == i])
    rmse, mape = Predict_LSTM(df[df['sku'] == i], i)
    Results_LSTM = Results_LSTM.append({'sku': str(i), 'rmse': rmse, 'mape': mape}, ignore_index=True)

#Font: https://medium.com/geekculture/time-series-forecast-using-deep-learning-adef5753ec85

## 3. Evaluación de la calidad de los modelos

#### A continuación usaremos los modelos creados y las previsiones hechas, para evaluar la calidad de los distintos modelos.



In [ ]:

Results1 = Results_SE[['sku','rmse','mape']]
Results1 = Results1.rename(columns={"rmse": "rmse_SE", "mape":"mape_SE"})

Results2 = Results_ARMA[['sku','rmse','mape']]
Results2 = Results2.rename(columns={"rmse": "rmse_ARMA", "mape":"mape_ARMA"})

Results3 = Results_SVR[['sku','rmse','mape']]
Results3 = Results3.rename(columns={"rmse": "rmse_SVR", "mape":"mape_SVR"})

Results4 = Results_LSTM[['sku','rmse','mape']]
Results4 = Results4.rename(columns={"rmse": "rmse_LSTM", "mape":"mape_LSTM"})

Results = pd.merge(Results1, Results2, on=['sku'], how='left')
Results = pd.merge(Results, Results3, on=['sku'], how='left')
Results = pd.merge(Results, Results4, on=['sku'], how='left')

Results


In [ ]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt

# RSME
se = Results[['rmse_SE']].mean()
arma = Results[['rmse_ARMA']].mean()
svr = Results[['rmse_SVR']].mean()
lstm = Results[['rmse_LSTM']].mean()

data = [se, arma, svr, lstm]
labels = ['se','arma','svr','lstm']
list = pd.DataFrame({'x':[labels], 'y':[data],}, columns=['x','y'])
p = sns.barplot(x=labels, y=data, data=list, ci=None).set(title='RMSE')



In [ ]:
#Valores medio:

print("RMSE (valor medio SE) = "+ str(se))

print("RMSE (valor medio ARMA) = "+ str(arma)) 

print("RMSE (valor medio SVR) = "+ str(svr)) 

print("RMSE (valor medio LSTM) = "+ str(lstm)) 

In [ ]:
#MAPE
se = Results[['mape_SE']].mean()
arma = Results[['mape_ARMA']].mean()
svr = Results[['mape_SVR']].mean()
lstm = Results[['mape_LSTM']].mean()

data = [se, arma, svr, lstm]
labels = ['se','arma','svr','lstm']

list = pd.DataFrame({'x':[labels], 'y':[data],}, columns=['x','y'])
p = sns.barplot(x=labels, y=data, data=list, ci=None).set(title='MAPE')

In [ ]:
#Valores medio:

print("RMSE (valor medio SE) = "+ str(se))

print("RMSE (valor medio ARMA) = "+ str(arma)) 

print("RMSE (valor medio SVR) = "+ str(svr)) 

print("RMSE (valor medio LSTM) = "+ str(lstm)) 